# Solving the 1D fluid flow problem using PINNs (Physics Informed Neural Networks)

## Problem Statement

let us have the following problem

$$
\frac{\partial}{\partial x}\left(\frac{\beta_c k A}{\mu B} \frac{\partial p}{\partial x}\right) \Delta x+q_{s c}=\frac{V_b \phi c_t}{\alpha_c B} \frac{\partial p}{\partial t}
$$

Assume that the viscosity and formation volume factor are constant and using the following data: Length $=2500 \mathrm{ft}$, Width $=80 \mathrm{ft}$, thickness $=60 \mathrm{ft} \quad B=1.2 \mathrm{bbl} / \mathrm{stb}, \alpha_c=5.615, \beta_c=1.127 \times 10^{-3}$ Well radius $r_w=0.33 \mathrm{ft}, \quad$ Viscosity $\mu=1.3 \mathrm{cpt}=1.3 \mathrm{cp}$

Total compressibility $c_t=7.3 \times 10^{-6} \mathrm{psi}^{-1} \quad$ Initial pressure $p_i=5000 \mathrm{psi}$ in all grid blocks Constant injection rate $700 \mathrm{stb} /$ day from Grid \# 2 . Constant production rate of $900 \mathrm{stb} /$ day at Grid \# 4 . There is a leakage of 250bbl/day of fluid at the left boundary $(x=0)$ and the pressure is held constant at the right boundary $(x=L)$ at 5200 psi and time-step size is 0.2 day. The table below gives the properties of each grid block:

### Setting the variables and constants

Let us have
$$ \Gamma(x) = \dfrac{\beta_c A k}{\mu B}
\quad \& \quad c\,(x) = \dfrac{V_b c_t \phi}{\alpha_c B}$$
So out problem can be written as
 $$
\frac{\partial}{\partial x} \left(\Gamma(x) \frac{\partial p}{\partial x}\right) \Delta x+q_{s c}= c\,(x) \frac{\partial p}{\partial t}
$$

Subject to the following boundary conditions

$$
q_{x0} = \left( \frac{\beta_c k A}{\mu} \frac{\partial p}{\partial x} \right)_{x=0, t}
\qquad p(x=L, t) = 5200
$$


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# set the device to be a GPU, if it is there
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# Defining constants

In [3]:
total_length = 10000
num_grids = 100
p_init = 5000.0
dy = 80.0 #width of a grid
dz = 60.0 #thickness of a grid
dx = total_length / num_grids
B, alpha, beta = 1.2, 5.615, 1.127e-3 # conversion factors
c_t = 7.3e-6 #compressibility
q_x0 = -250.0 #left boundary leakage
p_xL = 5200.0 #right boundary pressure

In [5]:
area = dy * dz
volume = dx * dy * dz
viscosity = 1.3

## import the porosity and the permeability

In [6]:
#comment this cell if you are not on google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#comment this cell if you are not on google colab
#copy the files to the runtime environment
permeability_path = "/content/drive/MyDrive/PETE547_Project/fluid_flow/perm.csv"
porosity_path = "/content/drive/MyDrive/PETE547_Project/fluid_flow/porosity.csv"

%cp {permeability_path} ./
%cp {porosity_path} ./

In [8]:
# import the data from the files
#make sure the files are in the same directory
permeability = torch.tensor(np.loadtxt('perm.csv'), dtype=torch.float32)
porosity = torch.tensor(np.loadtxt("porosity.csv"), dtype=torch.float32 )

## Define the wells values

In [9]:
#Since we only have 10 wells, create zero tensor and modify it
q_sc = torch.zeros(num_grids, dtype=torch.float32)
q_sc[0], q_sc[12], q_sc[25], q_sc[37] = 400, -500, 700, -400
q_sc[46], q_sc[54], q_sc[61] = -650, 1100, -800
q_sc[77], q_sc[84], q_sc[99] = 900, -450, 350
q_sc = q_sc.unsqueeze(-1)

### Setting the NEural network

In [10]:
# the model
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
            nn.Linear(N_INPUT, N_HIDDEN),
            activation()

        ])


        self.fch =nn.Sequential(*[ nn.Sequential(*[
            nn.Linear(N_HIDDEN, N_HIDDEN),
            activation()

        ]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN,N_OUTPUT)
        def weights_initialization(self):
                """"
                When we define all the modules such as the layers in '__init__()'
                method above, these are all stored in 'self.modules()'.
                We go through each module one by one. This is the entire network,
                basically.
                """
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight,gain=1.0)
                nn.init.constant_(m.bias, 0)
    def forward(self, x,t):
        inputs = torch.cat([x,t], axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        inputs = self.fcs(inputs)
        inputs = self.fch(inputs)
        inputs = self.fce(inputs)
        return inputs

# Generate Input Data

Since we are given the discretization for the length, which is 100 grids or control volumes, we can take one point from each one of the 100 grids. It is probably better that we avoid the interfaces between the grids such as $x= 0, 100, 200, ..., 1000$ since the values of the porosity and the permeability are given for a certain cell, but the interface point is between two cells or grids. An intuitive option is to take the point at the center of each grid such as $x = 50, 150, 150, ..., 950$  

**Note**: We can take more than one point from each control volume but we have to be careful with the porosity and the permability values. Any two points within the same grid will have the same porosity and permeability.

In [11]:
x_0, x_f = 0.0, total_length
t_0, t_f = 0.0, 10.0 #expriment on small data set

x_axis = torch.arange(dx/2, total_length, dx) #Boundary are excluded
t_axis = torch.linspace(0, t_f, 11)[1:] #Exaclude 0

#interior points
x_physics, t_physics = torch.split(torch.cartesian_prod(x_axis, t_axis), 1, 1)
porosity_physics = porosity.repeat_interleave(len(t_axis)).unsqueeze(-1)
permeability_physics = permeability.repeat_interleave(len(t_axis)).unsqueeze(-1)
q_sc_physics = q_sc.repeat_interleave(len(t_axis)).unsqueeze(-1)
gamma = ( beta * area * permeability_physics ) / ( viscosity * B )
c = ( volume * c_t * porosity_physics ) / ( alpha * B )

#initial points
x_init, t_init = torch.split(torch.cartesian_prod(x_axis, torch.tensor([t_0])), 1, 1)

#left boundary
x_left = torch.tensor( [x_0] * len(t_axis) ).unsqueeze(-1)

#right boundary
x_right = torch.tensor( [x_f] * len(t_axis) ).unsqueeze(-1)

#Track grad of some variables
x_physics = x_physics.requires_grad_(True)
t_physics = t_physics.requires_grad_(True)
x_left = x_left.requires_grad_(True)
x_axis = x_axis.unsqueeze(-1).requires_grad_(True)
t_axis = t_axis.unsqueeze(-1).requires_grad_(True)

In [12]:
pinn = FCN(2,1,32,5)
mse_cost_function = torch.nn.MSELoss()
optimizer = optim.Adam(pinn.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.45)

In [13]:
total_losses = []
iteration = []
boundary_loss = []
epochs = 10000

pinn.train()
for epoch in range(1, epochs+1):
    optimizer.zero_grad()

    #initial value loss
    p = pinn(x_init, t_init)
    initial_val_loss = torch.mean( ( p - p_init )**2 )

    #left boundary loss
    p = pinn(x_left, t_axis)
    p_x = torch.autograd.grad(p, x_left, torch.ones_like(p), create_graph=True)[0]
    left_boundary_loss = torch.mean( ( q_x0 -  B * gamma[0].item() * p_x )**2 )

    #right boundary loss
    p = pinn(x_right, t_axis)
    right_boundary_loss = torch.mean( ( p - p_xL )**2 )

    #pde loss
    p = pinn(x_physics, t_physics)
    p_x = torch.autograd.grad(p, x_physics, torch.ones_like(p), create_graph=True)[0]
    p_t = torch.autograd.grad(p, t_physics, torch.ones_like(p), create_graph=True)[0]
    p_x_gamma = p_x * gamma
    p_xx = torch.autograd.grad(p_x_gamma, x_physics, torch.ones_like(p_x_gamma), create_graph=True)[0]
    pde_loss = torch.mean( ( p_xx * dx + q_sc_physics - c * p_t )**2 )

    loss = initial_val_loss + left_boundary_loss + right_boundary_loss + pde_loss
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
      pinn.eval()
      with torch.inference_mode():
        print(f"init_loss: {initial_val_loss}, left: {left_boundary_loss}, right: {right_boundary_loss}, pde:{pde_loss}")
      pinn.train()

init_loss: 24588518.0, left: 0.8146690130233765, right: 26611982.0, pde:43177.12890625
